In [22]:
from migen import *
from migen.genlib.cdc import GrayDecoder

class FrequencyCounter(Module):
    def __init__(self, width):
        
        # set up a "beatnote clock domain"
        # allows you to add always @ statements
        self.clock_domains.bn = ClockDomain(reset_less=True)
        
        # gray counter modified from migen.genlib.cdc
        # advantage of using gray counter instead of binary counter is
        # only one bit will flip at a time; for crossing clock domains (ie, bringing
        # result into sys_clk) this avoids crazy transient errors where some counter bits
        # but not others have finished transitioning to their new state.
        self.q = Signal(width)
        self.q_next = Signal(width)
        self.q_binary = Signal(width)
        self.q_next_binary = Signal(width)
        
        
        # # #
        
        self.comb += [
            self.q_next_binary.eq(self.q_binary + 1),
            self.q_next.eq(self.q_next_binary ^ self.q_next_binary[1:])
        ]
        
        self.sync.bn += [
            self.q_binary.eq(self.q_next_binary),
            self.q.eq(self.q_next)
        ]
        
        # adding this to submodules as named attribute exposes self.decoder attribute
        self.submodules.decoder = GrayDecoder(width)
        self.comb += self.decoder.i.eq(self.q)


In [24]:
f = FrequencyCounter(3)
def tb(dut):
    for i in range(2**3+1):
        #yield dut.bn.clk.eq(~dut.bn.clk)
        yield dut.bn.clk
        yield

run_simulation(f, tb(f), vcd_name="grey.vcd")